- `pip install beautifulsoup4`
- `pip install lxml`

# get salary data

In [228]:
import requests
from bs4 import BeautifulSoup
import re
import ast
import pandas as pd

In [229]:
base_url = 'http://www.statiz.co.kr/salary.php?opt=0&sopt=2018&cnv=&pos=&te='
columns = ['선수명', '년도', '팀명', '연봉(2018)', 'WAR']
result_df = pd.DataFrame(columns=columns)

for idx in range(1,11):
    target_url = base_url + str(idx)
    req = requests.get(target_url)
    html = req.content
    soup = BeautifulSoup(html, 'lxml')
    salary_table = soup.find('table', attrs={'class':'table table-striped'})
    salary_table_row = salary_table.find_all('tr')[1:]

    for tr in salary_table_row:
        tds = tr.find_all('td')
        row = []
        for td in tds:
            row.append(td.text.upper())
        series = pd.Series(row, index=result_df.columns)
        result_df = result_df.append(series, ignore_index=True)

In [230]:
result_df = result_df[['선수명', '팀명', '연봉(2018)']]

In [231]:
# result_df.to_csv("baseball_player_salary_2018.csv", index=False, header=True)

# get player stats

In [232]:
df_batter = pd.read_csv("투수.csv")
df_pitcher = pd.read_csv("toosoo.csv")
df_pitcher.shape

(239, 21)

In [233]:
df_batter['팀명'] = df_batter['팀명'].apply(lambda x: x.upper())
del df_batter['#']
df_batter.shape

(292, 19)

In [234]:
df_pitcher['팀명'] = df_pitcher['팀명'].apply(lambda x: x.upper())
del df_pitcher['#']
df_pitcher.head()

,선수명,팀명,승,패,세,홀드,블론,경기,선발,이닝,삼진/9,볼넷/9,홈런/9,BABIP,LOB%,ERA,RA9-WAR,FIP,kFIP,WAR
0,켈리,SK,16,7,0,0,0,30,30,190.0,8.95,2.13,0.76,0.342,73.7,3.6,6.91,3.69,3.44,6.62
1,소사,LG,11,11,1,0,0,30,29,185.1,7.43,1.85,0.53,0.319,67.1,3.88,6.80,3.52,3.41,6.08
2,양현종,KIA,20,6,0,0,0,31,31,193.1,7.36,2.09,0.79,0.332,72.1,3.44,6.54,3.94,3.82,5.64
3,차우찬,LG,10,7,0,0,0,28,28,175.2,8.04,1.95,1.02,0.298,75.0,3.43,6.11,4.2,4.03,4.63
4,허프,LG,6,4,0,0,0,19,17,124.2,7.07,1.01,0.51,0.255,74.4,2.38,6.03,3.28,3.19,4.54


In [235]:
batter_list = df_batter['선수명'].unique().tolist()
pitcher_list = df_pitcher['선수명'].unique().tolist()

In [236]:
df_pitcher = df_pitcher[(df_pitcher['선수명'] != '김상수') & (df_pitcher['선수명'] != '김대우')]
df_batter = df_batter[(df_batter['선수명'] != '김상수') & (df_batter['선수명'] != '김대우')]

# merge

In [237]:
df_pitcher = df_pitcher.merge(result_df, on=['선수명', '팀명'], how='left')
df_batter = df_batter.merge(result_df, on=['선수명', '팀명'], how='left')

In [238]:
df_pitcher.head()

,선수명,팀명,승,패,세,홀드,블론,경기,선발,이닝,...,볼넷/9,홈런/9,BABIP,LOB%,ERA,RA9-WAR,FIP,kFIP,WAR,연봉(2018)
0,켈리,SK,16,7,0,0,0,30,30,190.0,...,2.13,0.76,0.342,73.7,3.6,6.91,3.69,3.44,6.62,"140,000"
1,소사,LG,11,11,1,0,0,30,29,185.1,...,1.85,0.53,0.319,67.1,3.88,6.80,3.52,3.41,6.08,"120,000"
2,양현종,KIA,20,6,0,0,0,31,31,193.1,...,2.09,0.79,0.332,72.1,3.44,6.54,3.94,3.82,5.64,"230,000"
3,차우찬,LG,10,7,0,0,0,28,28,175.2,...,1.95,1.02,0.298,75.0,3.43,6.11,4.2,4.03,4.63,"100,000"
4,허프,LG,6,4,0,0,0,19,17,124.2,...,1.01,0.51,0.255,74.4,2.38,6.03,3.28,3.19,4.54,NaN


In [239]:
df_batter.head()

,선수명,팀명,경기,타석,타수,안타,홈런,득점,타점,볼넷,삼진,도루,BABIP,타율,출루율,장타율,OPS,wOBA,WAR,연봉(2018)
0,최정,SK,130,527,430,136,46,89,113,70,107,1,0.316,0.316,0.427,0.684,1.111,0.442,7.3,"120,000"
1,김재환*,두산,144,636,544,185,35,110,115,81,123,4,0.385,0.34,0.429,0.603,1.032,0.427,7.22,NaN
2,최형우,KIA,142,629,514,176,26,98,120,96,82,0,0.362,0.342,0.45,0.576,1.026,0.43,7.2,"150,000"
3,박건우,두산,131,543,483,177,20,91,78,41,64,20,0.39,0.366,0.424,0.582,1.006,0.424,7.04,"37,000"
4,로사리오,한화,119,510,445,151,37,100,111,50,61,10,0.324,0.339,0.414,0.661,1.075,0.436,5.75,NaN


In [240]:
df_pitcher = df_pitcher.dropna()
df_batter = df_batter.dropna()

In [241]:
df_pitcher['연봉(2018)'] = df_pitcher['연봉(2018)'].apply(lambda x: x.replace(",", ""))
df_batter['연봉(2018)'] = df_batter['연봉(2018)'].apply(lambda x: x.replace(",", ""))
df_pitcher['연봉(2018)'] = df_pitcher['연봉(2018)'].astype(int)
df_batter['연봉(2018)'] = df_batter['연봉(2018)'].astype(int)

In [242]:
df_batter = df_batter[df_batter['선수명'] != '홍재호']
df_batter = df_batter[df_batter['선수명'] != '황경태']
df_batter = df_batter[df_batter['선수명'] != '김찬형']
df_batter = df_batter[df_batter['선수명'] != '강동관']

In [243]:
df_pitcher.to_csv("picher_stats_2017.csv", index=False, header=True)

In [244]:
df_batter.to_csv("batter_stats_2017.csv", index=False, header=True)

In [245]:
base_url = 'http://www.statiz.co.kr/salary.php?opt=0&sopt=2017&cnv=&pos=&te='
columns = ['선수명', '년도', '팀명', '연봉(2017)', 'WAR']
result_df = pd.DataFrame(columns=columns)

for idx in range(1,11):
    target_url = base_url + str(idx)
    req = requests.get(target_url)
    html = req.content
    soup = BeautifulSoup(html, 'lxml')
    salary_table = soup.find('table', attrs={'class':'table table-striped'})
    salary_table_row = salary_table.find_all('tr')[1:]

    for tr in salary_table_row:
        tds = tr.find_all('td')
        row = []
        for td in tds:
            row.append(td.text.upper())
        series = pd.Series(row, index=result_df.columns)
        result_df = result_df.append(series, ignore_index=True)

In [246]:
result_df = result_df[['선수명', '팀명', '연봉(2017)']]

In [247]:
df_pitcher = df_pitcher.merge(result_df, on=['선수명', '팀명'], how='left')
df_batter = df_batter.merge(result_df, on=['선수명', '팀명'], how='left')

In [248]:
df_batter = df_batter.dropna()
df_pitcher = df_pitcher.dropna()

In [249]:
df_pitcher['연봉(2017)'] = df_pitcher['연봉(2017)'].apply(lambda x: str(x).replace(",", ""))
df_batter['연봉(2017)'] = df_batter['연봉(2017)'].apply(lambda x: str(x).replace(",", ""))
df_pitcher['연봉(2017)'] = df_pitcher['연봉(2017)'].astype(int)
df_batter['연봉(2017)'] = df_batter['연봉(2017)'].astype(int)

In [250]:
df_batter.head()

,선수명,팀명,경기,타석,타수,안타,홈런,득점,타점,볼넷,...,도루,BABIP,타율,출루율,장타율,OPS,wOBA,WAR,연봉(2018),연봉(2017)
0,최정,SK,130,527,430,136,46,89,113,70,...,1,0.316,0.316,0.427,0.684,1.111,0.442,7.3,120000,120000
1,최형우,KIA,142,629,514,176,26,98,120,96,...,0,0.362,0.342,0.45,0.576,1.026,0.43,7.2,150000,150000
2,박건우,두산,131,543,483,177,20,91,78,41,...,20,0.39,0.366,0.424,0.582,1.006,0.424,7.04,37000,19500
3,나성범,NC,125,561,498,173,24,103,99,48,...,17,0.413,0.347,0.415,0.584,0.999,0.416,5.64,43000,35000
4,손아섭,롯데,144,667,576,193,20,113,80,83,...,25,0.374,0.335,0.42,0.514,0.934,0.398,5.6,150000,65000


In [251]:
df_pitcher.to_csv("picher_stats_2017.csv", index=False, header=True)
df_batter.to_csv("batter_stats_2017.csv", index=False, header=True)